# 基础准备工作

In [ ]:
# 1. 在cmd中，依次执行以下两行命令，创建并激活Python环境

# 使用canda创建新的Python环境
conda create -n rag_eval python=3.10

# 使用canda激活刚建的Python环境
conda activate rag_eval

In [ ]:
# 2. 安装项目的依赖包

pip install -r requirements.txt

In [ ]:
# 3. 添加jupyter notebook内核，添加完毕后，重启jupyter notebook

python -m ipykernel install --user --name=rag_eval

In [ ]:
# 4. 下载m3e-base向量模型到本地（共1.52GB，暂不支持API调用向量模型）
# 参考魔搭社区：https://www.modelscope.cn/models/xrunda/m3e-base/files

git lfs install
git clone https://www.modelscope.cn/xrunda/m3e-base.git

In [ ]:
# 5. 申请并创建智谱AI的接口

参考基础教程，具体路径：C2 使用 LLM API 开发应用==>2. 使用 LLM API.ipynb==>4. 使用智谱 GLM

In [ ]:
# 6. 添加配置文件

在项目文件夹的.env文件中填入申请的ZHIPUAI_API_KEY

In [1]:
# 7. 读取配置文件，加载环境变量

import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。
# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
_ = load_dotenv(find_dotenv())

assert os.environ["ZHIPUAI_API_KEY"] != "", "未填写ZHIPUAI_API_KEY"

In [2]:
# 8. 加载样例知识库文件

from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["eBook-How-to-Build-a-Career-in-AI.pdf"]).load_data()

In [3]:
# 9. 查看知识库文件解析内容

print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]), "\n")
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'> 

Doc ID: f8b4fd64-0363-4595-8553-518c9d552263
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


# 完整的RAG流程

In [4]:
# 1. 合并数据集，提取纯文本，便于后续使用

from llama_index.core import Document

document = Document(text="\n\n".join(doc.text for doc in documents))

In [5]:
# 2. 定义模型接口，参考官网教程：https://open.bigmodel.cn/dev/api/thirdparty-frame/langchain-sdk

from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext, load_index_from_storage, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.01,
    model="glm-3-turbo",
    openai_api_key=os.environ["ZHIPUAI_API_KEY"],
    openai_api_base="https://open.bigmodel.cn/api/paas/v4"
)

In [6]:
# 3. 定义并验证向量模型查询引擎，其中embed_model配置向量模型的实际存储位置

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:./m3e-base/")

index = VectorStoreIndex.from_documents([document], service_context=service_context)

query_engine = index.as_query_engine() 

response = query_engine.query("What are steps to take when finding projects to build your experience?") 

# 如果有正常输出，表明基本RAG环境可以正常运行
print(str(response))

C:\Users\fr\AppData\Local\Temp\ipykernel_3444\3042272740.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:./m3e-base/")
D:\workplace\Anaconda3\envs\rag_eval\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


To find projects that will build your experience, start by joining existing projects where you can contribute and learn. Engage in continuous reading, take courses, and discuss ideas with domain experts to generate new project ideas. Focus on application areas where machine learning has not yet been extensively applied. Develop side hustles or projects that can stir your creative juices and potentially lead to bigger opportunities. Look for projects that can help you grow technically, have good teammates, and can serve as stepping stones to larger projects. When presented with project ideas, consider factors such as cost of execution and the need for prior study. Adopt a collaborative approach to tackle large projects and work on developing your interpersonal and communication skills. Build a professional network and consider alternative approaches like community building instead of traditional networking. Remember that finding a job is just one step in your career journey. Cultivate p

# 配置使用TruLens

In [7]:
# 1. 准备验证测试问题，提前在eval_questions.txt文件中添加问题，一行表示一个问题

eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)
        print(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [8]:
# 2. Tru初始化，如果提示nltk下载失败，一般是网络问题
# 解决办法是将项目路径下的 nltk_cache.zip 解压到Python环境的 llama_index 目录中
# 例如：Anaconda的安装路径为：D:\Anaconda3，则解压到D:\Anaconda3\envs\rag_eval\Lib\site-packages\llama_index\legacy\_static

from utils import get_prebuilt_trulens_recorder
from trulens_eval import Tru
tru = Tru()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [9]:
# 3.重置Tru的统计数据库

tru.reset_database()
tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

In [10]:
# 4. 开始评估，自动记录评估指标，为了快速跑通流程，可以仅使用部分测试问题验证

with tru_recorder as recording:
    for question in eval_questions[:3]:
        response = query_engine.query(question)
        print(question, "-->", str(response), "\n")

What are the keys to building a career in AI? --> The keys to building a career in AI include learning foundational technical skills, working on projects to deepen those skills, building a portfolio to showcase progress, and using a framework for starting an AI job search. Additionally, overcoming imposter syndrome and making every day count are important aspects of a successful AI career. 



D:\workplace\Anaconda3\envs\rag_eval\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


How can teamwork contribute to success in AI? --> Teamwork contributes to success in AI by enabling the pooling of diverse skills and knowledge. It allows for the specialization of tasks, with each team member focusing on their area of expertise, which can lead to more efficient problem-solving and innovation. Collaboration also facilitates the exchange of ideas, leading to the exploration of new approaches and the improvement of existing solutions. Additionally, teamwork provides support and accountability, helping to maintain momentum and overcome challenges throughout the project lifecycle. 

What is the importance of networking in AI? --> Networking is crucial in AI as it enables the exchange of ideas, knowledge, and resources among professionals. It helps in building relationships with peers, mentors, and industry experts, who can provide guidance, support, and collaboration opportunities. Networking also allows individuals to stay updated with the latest trends, research, and bes

# 预览评估结果

In [11]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
Direct Query Engine,0.7,0.8,1.0,10.0,0.0


In [12]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [13]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_76ebc30902ecff6cbd53c2b83d89e7a4,"""What are the keys to building a career in AI?""","""The keys to building a career in AI include l...",-,"{""record_id"": ""record_hash_76ebc30902ecff6cbd5...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-10T21:43:52.893627"", ""...",2025-01-10T21:44:02.931425,1.0,0.7,1.0,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",10,2492,0.0
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_175e38d7adeb5be9c2f25fd242cefe24,"""How can teamwork contribute to success in AI?""","""Teamwork contributes to success in AI by enab...",-,"{""record_id"": ""record_hash_175e38d7adeb5be9c2f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-10T21:44:03.707803"", ""...",2025-01-10T21:44:13.770304,1.0,0.7,0.6,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Class projects: 3. Creat...,10,2392,0.0
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_978c82540e9ce0a2f773943286489dcc,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it enables the...",-,"{""record_id"": ""record_hash_978c82540e9ce0a2f77...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-01-10T21:44:14.528838"", ""...",2025-01-10T21:44:24.615395,NaN,NaN,NaN,NaN,NaN,NaN,10,2447,0.0
